In [1]:
using BenchmarkTools: @ballocated
using LinearAlgebra: I, norm, triu, tril, tr, diagm, diag, qr, eigvals
using CairoMakie

In [2]:
function hessenberg_form!(T)
    # based on Trefethen pg 198
    m = size(T, 1)
    for k = 1:m-2
        vk = T[k+1:m, k]

        save_norm = sign(vk[1])*norm(vk)
        vk[1] += sign(vk[1])*norm(vk)

        vk /= norm(vk)

        T[k+1:m, k:m] -= 2 * vk * vk' * T[k+1:m, k:m]
        T[1:m, k+1:m] -= 2 * T[1:m,k+1:m] * vk * vk'

        T[k+1, k] = -save_norm

        for i = k+2:m
            T[i,k] = 0.0
        end
    end
end

# make symmetric matrix
m = 5
T = rand(Float64, m, m)
T = T'T
# display(T)
traceA = tr(T)
# display(T)
display(eigvals(T))
hessenberg_form!(T)
# display(T)
display(eigvals(T))

# Checks
# + Trace?
# + close enough to zero?
# - doesn't allocates memory?
@assert tr(T) ≈ traceA
@assert sum(tril(T,-2)) ≈ 0
# T = randn(5, 5)
# allocated_memory = @ballocated  hessenberg_form!(T)
# print(allocated_memory)
# @assert allocated_memory == 0
display(T)

5-element Vector{Float64}:
 0.0010289134448810923
 0.07206498075576494
 0.37106612963089847
 1.0532711264373162
 6.737922824287816

5-element Vector{Float64}:
 0.001028913444881368
 0.0720649807557652
 0.37106612963089886
 1.053271126437315
 6.737922824287812

5×5 Matrix{Float64}:
  1.3337   -2.44907   -6.32084e-16  -2.09582e-16  -1.67053e-17
 -2.44907   5.49704   -0.867547     -5.63536e-17  -4.43983e-16
  0.0      -0.867547   0.991459      0.11996       6.93889e-17
  0.0       0.0        0.11996       0.191717      0.16879
  0.0       0.0        0.0           0.16879       0.221438

In [3]:
function givens_qr!(T)
    m = size(T, 1)
    for k = 1:m-1
        # make givens matrix
        G = zeros(m, m)
        u = [T[k,k]; T[k+1, k]]

        if u[2] == 0
            c = 1
            s = 0
        else
            if abs(u[2]) > abs(u[1])
                tau = -u[1]/u[2]
                s = 1/sqrt(1+tau*tau)
                c = s*tau
            else
                tau = -u[2]/u[1]
                c = 1/sqrt(1+tau*tau)
                s = c*tau
            end
        end
    
        G = [c s; -s c]

        T[:, k:k+1] = T[:, k:k+1] * G
        T[k:k+1, :] = G' * T[k:k+1, :]
    end
end

m = 5
T = rand(m,m)
T = T'T
hessenberg_form!(T)

Q,R = qr(T)
RQ = R*Q

givens_qr!(T)
# display(T)
# display(RQ)
@assert abs.(T) ≈ abs.(RQ)

In [58]:
function practical_QR_with_shifts!(T, shift)
    tol = 1e-5
    m = size(T, 1)

    while true
        min_val = Inf
        min_val_idx = -1
        for i=1:m-1
            if (abs(T[i+1, i]) < min_val)
                min_val = abs(T[i+1, i])
                min_val_idx = i
            end
        end

        if min_val < tol 
            if min_val_idx != 1
                # println("bounds: 1, ", min_val_idx)
                practical_QR_with_shifts!(@view(T[1:min_val_idx, 1:min_val_idx]), shift)
            end
            if min_val_idx != m-1
                # println("bounds: ", min_val_idx+1, " , ", m)
                practical_QR_with_shifts!(@view(T[min_val_idx+1:m, min_val_idx+1:m]), shift)
            end
            break
        else
            if shift == "single"
                mu = T[m, m]
            elseif shift == "wilkinson"
                # Implemented from Trefethen 222
                B = T[m-1:m,m-1:m]
                
                delta = (B[1,1] - B[2,2] )/2
                del_sgn = sign(delta)
                if delta == 0
                    del_sgn = 1
                end
                mu = B[2, 2] - del_sgn * B[1, 1]^2 / (abs(delta) + sqrt(delta^2 + B[1, 2]^2))
            end

            for i=1:m
                T[i,i] -= mu 
            end
            givens_qr!(T)
            for i=1:m
                T[i,i] += mu
            end
        end
    end
end

m = 5
A = rand(m,m)
Q,_ = qr(A)
λ = 3 .^ range(0,m-1)
Σ = diagm(λ)
T = Q'Σ*Q
hessenberg_form!(T)

shift = "single"
practical_QR_with_shifts!(T, shift)
display(T)

@assert λ ≈ sort(diag(T))

5×5 Matrix{Float64}:
 81.0          -1.11272e-8    2.10037e-15  -2.37638e-15   3.85796e-16
 -1.11272e-8   27.0          -4.13594e-7   -8.63639e-15  -3.19941e-15
 -5.01004e-16  -4.13594e-7    9.0           7.85253e-8   -6.4912e-15
  1.69548e-17   5.24444e-16   7.85253e-8    1.0          -2.88599e-6
  7.98897e-17  -2.87669e-16   2.23227e-17  -2.88599e-6    3.0

In [181]:
function practical_QR_with_shifts_hist!(T, T_hist, T_conv, shift)
    tol = 1e-15
    m = size(T, 1)

    while true
        min_val = Inf
        min_val_idx = -1
        for i=1:m-1
            if (abs(T[i+1, i]) < min_val)
                min_val = abs(T[i+1, i])
                min_val_idx = i
            end
        end

        if min_val < tol 
            # println("min val", min_val_idx)
            if min_val_idx == 1
                push!(T_conv, T[min_val_idx, min_val_idx])
                # println("Tconv_1", T_conv)
            elseif min_val_idx == m-1
                push!(T_conv, T[min_val_idx+1, min_val_idx+1])
                # println("Tconv_m", T_conv)
            end

            if min_val_idx != 1
                # println("bounds: 1, ", min_val_idx)
                practical_QR_with_shifts_hist!(@view(T[1:min_val_idx, 1:min_val_idx]), T_hist, T_conv,  shift)
            end
            if min_val_idx != m-1
                # println("bounds: ", min_val_idx+1, " , ", m)
                practical_QR_with_shifts_hist!(@view(T[min_val_idx+1:m, min_val_idx+1:m]), T_hist, T_conv, shift)
            end
            break
        else
            if shift == "single"
                mu = T[m, m]
            elseif shift == "wilkinson"
                # Implemented from Trefethen 222
                B = T[m-1:m,m-1:m]
                
                delta = (B[1,1] - B[2,2] )/2
                del_sgn = sign(delta)
                if delta == 0
                    del_sgn = 1
                end
                mu = B[2, 2] - del_sgn * B[1, 1]^2 / (abs(delta) + sqrt(delta^2 + B[1, 2]^2))
            end

            for i=1:m
                T[i,i] -= mu 
            end
            givens_qr!(T)
            for i=1:m
                T[i,i] += mu
            end
            
            push!(T_hist, diag(T))
            # display(T_hist[end])
            # display(T_conv)
            T_hist[end] = append!(T_hist[end], T_conv)
            # global_itr += 1
            # display(T_hist)
        end
    end
end

m = 16
A = rand(m,m)
Q,_ = qr(A)
λ = 3 .^ range(0,m-1)
Σ = diagm(λ)
T = Q'Σ*Q
hessenberg_form!(T)

T_hist = []
T_conv = []
global_itr = 1
shift = "single"
practical_QR_with_shifts_hist!(T, T_hist, T_conv, shift)
display(T)
# println(sort(diag(T)))
# println(sort(T_hist))
@assert λ ≈ sort(diag(T))

# max_eigs = Float64[]
# min_eigs = Float64[]
# for t in T_hist
#     sort!(t)
#     push!(max_eigs, t[end])
#     push!(min_eigs, t[1])
# end

# max_eig_error = abs.(max_eigs .- λ[end])./λ[end]
# min_eig_error = abs.(min_eigs .- λ[1])./λ[1]

# error_plot = scatter(1:length(max_eigs), max_eig_error, label="max eig", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
# scatter!(1:length(max_eigs), min_eig_error, label="min_eig")
# axislegend(position=:rt)
# save("single.pdf", error_plot)

16×16 Matrix{Float64}:
  1.43489e7     2.96153e-10  -1.11683e-9   …  -1.99836e-9    1.50165e-9
  0.0           4.78297e6    -4.98191e-10      3.88053e-12  -8.55918e-11
  4.57135e-11   0.0           1.59432e6        3.72155e-10  -7.93873e-11
 -2.62691e-11  -5.13559e-12   0.0             -7.73388e-11   3.48507e-10
  9.77608e-11   4.39795e-12  -1.40733e-12      2.61485e-10  -2.04041e-10
  3.60878e-11  -4.54146e-12   1.30295e-12  …  -3.82519e-10  -3.56716e-10
 -8.54559e-12   1.862e-12    -3.67367e-13      7.40715e-11   4.95135e-10
  1.97862e-11  -5.03829e-12   7.52975e-13      3.63215e-10  -2.29713e-10
 -3.04305e-11   8.14579e-12  -1.0705e-12      -1.82977e-10  -7.72522e-10
 -5.05177e-11   1.37473e-11  -1.72075e-12      3.11883e-10   4.29345e-10
 -1.91271e-11   5.23361e-12  -6.4405e-13   …  -3.63606e-10  -1.92823e-10
 -4.7192e-12    1.29364e-12  -1.58281e-13     -1.13726e-10  -6.12511e-10
  2.29731e-11  -6.30206e-12   7.69456e-13      2.85848e-11   5.67586e-10
  3.43921e-11  -9.44688e-12  

In [163]:
m = 5
A = rand(m,m)
Q,_ = qr(A)
λ = 3 .^ range(0,m-1)
Σ = diagm(λ)
T = Q'Σ*Q
hessenberg_form!(T)

T_hist = []
T_conv = []
global_itr = 1
shift = "wilkinson"
practical_QR_with_shifts_hist!(T, T_hist, T_conv, shift)
display(T)
# println(sort(diag(T)))
# println(sort(T_hist))
@assert λ ≈ sort(diag(T))

max_eigs = Float64[]
min_eigs = Float64[]
for t in T_hist
    sort!(t)
    push!(max_eigs, t[end])
    push!(min_eigs, t[1])
end

max_eig_error = abs.(max_eigs .- λ[end])./λ[end]
min_eig_error = abs.(min_eigs .- λ[1])./λ[1]

error_plot = scatter(1:length(max_eigs), max_eig_error, label="max eig", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
scatter!(1:length(max_eigs), min_eig_error, label="min_eig")
axislegend(position=:rt)
save("wilkinson.pdf", error_plot)

# println(length.(T_hist))

5×5 Matrix{Float64}:
 81.0          -6.60426e-11   1.44858e-15  -1.79645e-15   2.28702e-15
 -6.60474e-11  27.0          -6.05106e-11   6.47551e-15  -3.86153e-16
  4.38078e-16  -6.05123e-11   9.0           8.39733e-11   2.38961e-15
 -2.15966e-16   2.41637e-17   8.39726e-11   3.0          -8.49946e-11
  8.40889e-18   2.82479e-17  -7.07842e-17  -8.49888e-11   1.0

CairoMakie.Screen{PDF}


In [182]:
m = 32
A = rand(m,m)
Q,_ = qr(A)
λ = 3 .^ range(0,m-1)
Σ = diagm(λ)
T = Q'Σ*Q
hessenberg_form!(T)
T_single = copy(T)
T_wilk = copy(T)

T_hist_single = []
T_conv_single = []
T_hist_wilk = []
T_conv_wilk = []

practical_QR_with_shifts_hist!(T_single, T_hist_single, T_conv_single, "single")
practical_QR_with_shifts_hist!(T_wilk, T_hist_wilk, T_conv_wilk, "wilkinson")

max_eigs_single = Float64[]
max_eigs_wilk = Float64[]

for t in T_hist_single
    sort!(t)
    push!(max_eigs_single, t[end])
    # push!(min_eigs, t[1])
end
for t in T_hist_wilk
    sort!(t)
    push!(max_eigs_wilk, t[end])
    # push!(min_eigs, t[1])
end

max_eig_error_single = abs.(max_eigs_single .- λ[end])./λ[end]
max_eig_error_wilk = abs.(max_eigs_wilk .- λ[end])./λ[end]


error_plot = scatter(1:length(max_eigs_single), max_eig_error_single, label="Single Shift", axis=(yscale=log10, xlabel="iteration, k", ylabel="Normalized Eigenvalue Error"))
scatter!(1:length(max_eig_error_wilk), max_eig_error_wilk, label="Wilkinson Shift")
xlims!(0, 30)
axislegend(position=:rt)
save("compare.pdf", error_plot)

CairoMakie.Screen{PDF}
